In [1]:
# let's fucking build GPT I guess!
#   - starting by going all the way back to the bigram lookup table model...
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.set_default_device("mps")  # use gpu
torch.manual_seed(1337)  # seed for reproducibility

In [2]:
# read tinyshakespeare dataset
with open('res/tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(text[:999])
print("length of dataset in characters: ", len(text))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [3]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encode: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decode: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [5]:
# it's worth understanding how LLMs perform tokenization:
#   - GPTs use Byte Pair Encoding (BPE) implementations (Google's SentencePiece, OpenAi's TikToken, etc.)
#   - BPE produces a vocabulary of tokens that can be whole words (like "the"), parts of words (like "ing"),
#     or even single characters, depending on what’s common in the data.
#   - it's clever because it handles rare words by breaking them into smaller chunks (like "unhappiness" →
#     "un" + "hap" + "pi" + "ness") and keeps frequent stuff compact.
#       - you can play around with tiktoken to see for yourself (either locally via pip install or online)

In [6]:
# let's now encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this
print(data.shape, data.dtype)

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [7]:
# let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [8]:
# Ex. single data "chunk": 8 tokens of context, 1 token to predict.
#   - the job of the model is to predict the next token given the context.
block_size = 8
print(train_data[:block_size+1])
print(train_data[:block_size+1].shape)

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58], device='mps:0')
torch.Size([9])


In [9]:
# Ex. we actually have 8 training samples in this chunk
#   - for every token `t` (besides the first), use the previous tokens as the context to predict `t`.
#   - our model can be trained to predict on all of these (i.e. make it learn variable context sizes).
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Ex{t+1}. when input is {context} the target is: {target}")

Ex1. when input is tensor([18], device='mps:0') the target is: 47
Ex2. when input is tensor([18, 47], device='mps:0') the target is: 56
Ex3. when input is tensor([18, 47, 56], device='mps:0') the target is: 57
Ex4. when input is tensor([18, 47, 56, 57], device='mps:0') the target is: 58
Ex5. when input is tensor([18, 47, 56, 57, 58], device='mps:0') the target is: 1
Ex6. when input is tensor([18, 47, 56, 57, 58,  1], device='mps:0') the target is: 15
Ex7. when input is tensor([18, 47, 56, 57, 58,  1, 15], device='mps:0') the target is: 47
Ex8. when input is tensor([18, 47, 56, 57, 58,  1, 15, 47], device='mps:0') the target is: 58


In [10]:
# create batches of data (inputs + targets) from either the train or validation set
#   - inputs will have shape (B,T) i.e. batch x time
#   - targets will also have shape (B,T) i.e. batch x time
#   - training objective: model(inputs[b, :t]) = targets[b, :t+1])
# pretty nice, compact representation! (very important to understand!)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 0, 32, 53,  1, 41, 39, 56, 60],
        [40, 50, 43, 57,  6,  1, 61, 46],
        [ 1, 54, 39, 58, 47, 43, 52, 41],
        [63,  1, 57, 61, 53, 56, 42,  6]], device='mps:0')
targets:
torch.Size([4, 8])
tensor([[32, 53,  1, 41, 39, 56, 60, 43],
        [50, 43, 57,  6,  1, 61, 46, 53],
        [54, 39, 58, 47, 43, 52, 41, 43],
        [ 1, 57, 61, 53, 56, 42,  6,  1]], device='mps:0')
----
when input is [0] the target: 32
when input is [0, 32] the target: 53
when input is [0, 32, 53] the target: 1
when input is [0, 32, 53, 1] the target: 41
when input is [0, 32, 53, 1, 41] the target: 39
when input is [0, 32, 53, 1, 41, 39] the target: 56
when input is [0, 32, 53, 1, 41, 39, 56] the target: 60
when input is [0, 32, 53, 1, 41, 39, 56, 60] the target: 43
when input is [40] the target: 50
when input is [40, 50] the target: 43
when input is [40, 50, 43] the target: 57
when input is [40, 50, 43, 57] the target: 6
when input is [40, 50, 43, 57, 6] the tar

In [11]:
# build a bigram language model, exactly the same as what we made in makemore p1
#   - see p1_1_bigram_neural_network.ipynb
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        #   - logit takes shape (B,T,C) i.e. batch x time x channel
        #   - batch indexes over the samples, time indexes over the tokens (context) in each
        #     sample, and channel indexes into the embedding vector of each token.
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([4, 8, 65])
tensor(4.7038, device='mps:0', grad_fn=<NllLossBackward0>)

aa&3;?MYMxZbEcqEPVQsReN
OkuUfZEieMqy:r,--rk,ECOIiZeJtEjn?u
;JurYhM'eTAwyFXPhjCTg&qof!zAZrFoBlyL:cL&P


In [12]:
# create an optimizer (we need to use this shitty hack - https://github.com/pytorch/pytorch/issues/149184)
torch.set_default_device("cpu")  # use cpu for a bit
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-2)
torch.set_default_device("mps")  # go back to gpu

In [13]:
# train the bigram model
batch_size = 32
for steps in range(1000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.414292573928833


In [14]:
# let's generate some samples
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


ORI luje omy heV:
O:
CELAnr core' venviforo-NT: rsw'll isurendime ilerveMfin?

BRI meso otthe pr w o can w'losto h prin hifacom! Soums masan IZBEHARomquchinout ik'sthar: hen rvaiwit, curencer y ve il
UERDI to welilearas be mpuresour,

Whatte'd aized'?

tar acanou tarts afeThopringheware Yowhare fofot foling a ge cr'farmG ce areaw bellathy feldin bard henoslendothecendeang--Gld! oNR buthet, miz! hat 'd:
se n woo ff f f PRI d

Wheck nuindeeaieet eng nd
OLAy hS:

Wher mowha INCThe ma!Hiprd s and, y
